In [19]:
import sqlite3
from conllu import parse
from collections import OrderedDict

input_text = 'Показать новую выставку приехал плодовитый российский мультимедийный арт-коллектив Electroboutique со своими уникальными device-art объектами.'

#здесь должна быть библиотека, но с ней некоторые проблемы
rusclasp_result = {'text': 'Показать новую выставку приехал плодовитый российский мультимедийный арт-коллектив Electroboutique со своими уникальными device-art объектами.', 'entities': [['T1', 'Span', [[0, 141]]]], 'relations': []}

#смотрим, как разделяет rusclasp
clauses_frames = []                    #рамки клауз
begins = []                            #индексы, где начинаются клаузы
ends = []                              #индексы, где заканчиваются клаузы
split_punc = []                        #индексы знаков препинания, по которым делит
rusclasp_clauses = []

entities = rusclasp_result['entities']
for section in entities:
  sec_num = section[2]
  clauses_frames.append(sec_num)
  begins.append(sec_num[0][0])
  ends.append(sec_num[0][1])
  split_punc.append(sec_num[0][1] + 1)
  for x in sec_num:
      clause = input_text[x[0]:x[1]]
      rusclasp_clauses.append(clause)

pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, Pipeline.DEFAULT)
parsed = pipeline.process(input_text)
my_example = str(parsed)
sentences = parse(my_example)
sentence = sentences[0]

id = [inf['id'] for inf in sentence]
form = [inf['form'] for inf in sentence]
lemma = [inf['lemma'] for inf in sentence]
upostag = [inf['upostag'] for inf in sentence]
feats = [inf['feats'] for inf in sentence]
head = [inf['head'] for inf in sentence]
deprel = [inf['deprel'] for inf in sentence]

begins_words = [0]                           #индекс начала каждого токена
ends_words = []                              #индекс окончания каждого токена
i=0
for x in input_text:
  if x ==' ':
    begins_words.append(i+1)
    ends_words.append(i)
  i += 1
  
points = ['.','!','?']
if input_text[len(input_text)-1] in points:
  ends_words.append(len(input_text)-1)
  ends_words.append(len(input_text))
  begins_words.append(len(input_text)-1)
else:
  ends_words.append(len(input_text))

conn = sqlite3.connect('polipredications.db')
c = conn.cursor()
c.execute("CREATE TABLE IF NOT EXISTS tokens(id, form, lemma, upostag, feats, head, deprel, begin, end, rcl_split)")
for i in range(len(id)):
  feat = feats[i]
  if feat == None:
    feat_i = 'None'
  else:
    feat_i = ','.join(list(feats[i].values()))
  if ends_words[i] in split_punc:
    rcl_split = 1
  else:
    rcl_split = 0
  if begins_words[i] in split_punc:
    rcl_split = 1
  c.execute("INSERT INTO tokens VALUES (?,?,?,?,?,?,?,?,?,?)", (id[i], form[i], lemma[i], upostag[i], feat_i, head[i], deprel[i], begins_words[i], ends_words[i], rcl_split))

conn.commit()



for row in c.execute('SELECT * FROM tokens ORDER BY id'):
  print(row)  

(1, 'Показать', 'показывать', 'VERB', 'Perf,Inf,Act', 4, 'nsubj', 0, 8, 0)
(2, 'новую', 'новый', 'ADJ', 'Acc,Pos,Fem,Sing', 3, 'amod', 9, 14, 0)
(3, 'выставку', 'выставка', 'NOUN', 'Inan,Acc,Fem,Sing', 1, 'obj', 15, 23, 0)
(4, 'приехал', 'приезжать', 'VERB', 'Perf,Masc,Ind,Sing,Past,Fin,Act', 0, 'root', 24, 31, 0)
(5, 'плодовитый', 'плодовитый', 'ADJ', 'Inan,Acc,Pos,Masc,Sing', 8, 'amod', 32, 42, 0)
(6, 'российский', 'российский', 'ADJ', 'Inan,Acc,Pos,Masc,Sing', 8, 'amod', 43, 53, 0)
(7, 'мультимедийный', 'мультимедийный', 'ADJ', 'Inan,Acc,Pos,Masc,Sing', 8, 'amod', 54, 68, 0)
(8, 'арт-коллектив', 'арт-коллектив', 'NOUN', 'Inan,Acc,Masc,Sing', 4, 'obj', 69, 82, 0)
(9, 'Electroboutique', 'Electroboutique', 'PROPN', 'Yes', 8, 'flat:foreign', 83, 98, 0)
(10, 'со', 'с', 'ADP', 'None', 14, 'case', 99, 101, 0)
(11, 'своими', 'свой', 'DET', 'Ins,Plur', 14, 'amod', 102, 108, 0)
(12, 'уникальными', 'уникальный', 'ADJ', 'Ins,Pos,Plur', 14, 'amod', 109, 120, 0)
(13, 'device-art', 'device-art', '

In [20]:
num_verbs = []
conn = sqlite3.connect('polipredications.db')
c = conn.cursor()
i = 0
for row in c.execute('SELECT * FROM tokens ORDER BY id'):
    pos = row[3]
    if 'VERB' in pos:
        i+=1
    if row[9] == 1:
        num_verbs.append(i)
        i=0
verb_clause = zip(num_verbs,clauses_frames)
needed_clauses = []                             #клаузы, в которых находятся финитный и нефинитный глаголы вместе
for x in verb_clause:
    if x[0] >= 2:
        needed_clauses.append(x[1])
      
with open('finit_verbs.txt','r', encoding='utf-8') as f:
    text =  f.read()
    for row in c.execute('SELECT * FROM tokens ORDER BY id'):
        for x in needed_clauses:
            if row[8] in range(x[0][0],x[0][1]):
                if 'Fin' in row[4]:                       
                    if row[2] not in text:                     # значит нефинитный глагол просто аргумент финитного
                        for cl in rusclasp_clauses:
                            print(cl)                   
                            break
                    else:
                      finit_row = row
                      frames.append(row[0])
                if 'Inf' in row[4]:
                    infinit_row = row
                    frames.append(row[0])

#определяем к чему отнести все то, что стоит между глаголами
frames = []

if finit_row[0] > infinit_row[0]:
    split_punc.append(finit_row[7])        #в таком случае синт.парсер всегда показывает зависимость он инфинитива, даже если это не так
        
                            
if finit_row[0] < infinit_row[0]:
    if infinit_row[0] == finit_row[0] + 1:
        split_punc.append(infinit_row[7])
    else:
        frames.append(finit_row[0]+1)
        frames.append(infinit_row[0])

# Выдаём правильные склеивания
split_punc.append(0)
split_punc = sorted(split_punc)
split_punc = split_punc[:-1]
x = map(lambda x: input_text[slice(*x)], zip(split_punc, split_punc[1:] + [None]))
for i in x:
  print('ЭДЕ: ',i)
print(split_punc)

ЭДЕ:  Показать новую выставку 
ЭДЕ:  приехал плодовитый российский мультимедийный арт-коллектив Electroboutique со своими уникальными device-art объектами.
[0, 24]
